In [243]:
import numpy as np
import pandas as pd

In [244]:
training_data = np.load(f'../fashion_train.npy')
test_data = np.load(f'../fashion_test.npy')

In [254]:
df = pd.DataFrame(training_data)
dftest = pd.DataFrame(test_data)


dftest_2classes =  dftest#[dftest.iloc[:, -1] < 2]
df_2classes = df#[df.iloc[:, -1] < 2]
df_2classes.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,3,0,0,0,41,...,63,78,11,0,2,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,32,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,1,0,0,0,...,168,119,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,98,...,21,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,2,0,0,...,88,128,46,0,0,0,0,0,0,4


### Create subset for fast testing

In [405]:
# Split data: 80% sample
df_2classes_subset = df_2classes.sample(frac=0.5, random_state=1)

# Separate the last column to keep it unchanged
#last_column = df_2classes_subset.iloc[:, -1]

# Apply transformation to all columns except the last one
#df_2classes_subset.iloc[:, :-1] = (df_2classes_subset.iloc[:, :-1] / 255 * 10).round().astype(int)

# Concatenate the transformed data and the last column
#df_2classes_subset = pd.concat([df_2classes_subset.iloc[:, :-1], last_column], axis=1)

#df_2classes_subset


In [336]:
# Split data: 80% sample
test_data_trans = df_2classes.sample(frac=1, random_state=1)

# Separate the last column to keep it unchanged
#last_column = test_data_trans.iloc[:, -1]

# Apply transformation to all columns except the last one
#test_data_trans.iloc[:, :-1] = (test_data_trans.iloc[:, :-1] / 255 * 10).round().astype(int)

# Concatenate the transformed data and the last column
#test_data_trans = pd.concat([test_data_trans.iloc[:, :-1], last_column], axis=1)

#test_data_trans


### Decision tree (Needs water)

In [248]:
def find_best_split(data):

    best_i=None
    best_column=None
    current_min = 2  # Gini ranges from 0 to 1; this initializes a high value.
    best_split1, best_split2 = None, None
    split_info = None    

    n = len(data)

    for column in data.columns[:-1]:
        # Sort data by the column for efficient split calculations
        sorted_data = data.sort_values(by=column)
        values = sorted_data[column].values
        labels = sorted_data[data.columns[-1]].values
 
        # Initialize running totals for calculating Gini efficiently
        left_counts = np.zeros_like(labels, dtype=float)
        right_counts = np.bincount(labels, minlength=2).astype(float)
        total_right = len(labels)
 
        for i in range(len(values) - 1):
            label = labels[i]
            left_counts[label] += 1
            right_counts[label] -= 1
 
            total_left = i + 1
            total_right -= 1
           
 
            # Only consider unique values for splits
            if values[i] == values[i + 1]:
                continue
 
            gini_left = 1.0 - np.sum((left_counts / total_left) ** 2)
            gini_right = 1.0 - np.sum((right_counts / total_right) ** 2)
            weighted_gini = (gini_left * total_left + gini_right * total_right) / n
            
 
            if weighted_gini < current_min:
                best_i=i
                best_column=column
                current_min = weighted_gini
                split_info = [current_min, column, values[i]]


    # Sort the data again by the best column
    sorted_data = data.sort_values(by=best_column)

    # Now split the data into two parts based on best_i
    best_split1 = sorted_data.iloc[:best_i + 1]  # Use iloc to select all columns up to best_i
    best_split2 = sorted_data.iloc[best_i + 1:]  # Use iloc to select all columns after best_i

    Split_majoritys=[(best_split1[data.columns[-1]].mode().iloc[0]),best_split2[data.columns[-1]].mode().iloc[0]]

    return split_info,Split_majoritys

In [406]:
k = 10
splits = {i: [] for i in range(k)}

def split_data(data,level):

    if  isinstance(data, str) and data == "deadleaf": #leaf detection
      splits[level].append(("deadleaf")) 
      if level + 1 < k:
            split_data("deadleaf", level + 1)
            split_data("deadleaf", level + 1)

    elif data.iloc[:, -1].nunique() == 1: #leaf detection
      majority=(data[data.columns[-1]].mode().iloc[0])
      splits[level].append(("leaf",majority)) 
      if level + 1 < k:
            split_data("deadleaf", level + 1)
            split_data("deadleaf", level + 1)

    else:
      result=find_best_split(data)
      gini,column,value=result[0]
      left_majority,right_majority=result[1]
      df1 = data[data[column] > value]
      df2 = data[data[column] <= value]
      # Update splits dictionary
      splits[level].append((gini,column,value,left_majority,right_majority))
      #splits[level].append((len(data), split_index))
      # Recursive splitting until level reaches `k`
      if level + 1 < k:
          split_data(df1, level + 1)
          split_data(df2, level + 1)


split_data(df_2classes_subset,0)


In [408]:
print(splits[0])
print(splits[1])
print(splits[2])
print(splits[3])
print(splits[4])
print(splits[5])
print(splits[6])
print(splits[7])
print(splits[8])
print(splits[9])


splits2=splits




[(0.6544374083701519, 546, 5, 1, 0)]
[(0.6255895417689575, 621, 9, 0, 2), (0.04820852485789488, 543, 54, 1, 1)]
[(0.41087954420312, 46, 12, 4, 2), (0.5013204508878081, 173, 8, 3, 0), (0.0051183431465122095, 602, 86, 1, 2), (0.6182881446039341, 751, 37, 1, 2)]
[(0.3685874801916122, 771, 29, 2, 2), (0.3100234867345764, 769, 4, 2, 4), (0.37689627690927846, 425, 19, 0, 4), (0.49794863925735994, 623, 5, 3, 3), (0.0, 16, 29, 4, 2), (0.0, 515, 26, 0, 1), (0.0, 5, 1, 2, 1), (0.5829307568438002, 235, 187, 1, 3)]
[(0.6083763192135873, 471, 42, 4, 2), (0.24290606757059727, 778, 6, 2, 2), (0.11761599159986523, 88, 7, 4, 0), (0.44065694865745425, 104, 35, 2, 4), (0.5361268526591108, 247, 11, 3, 4), (0.3028803415189921, 38, 25, 0, 0), (0.6056824955611729, 232, 4, 3, 0), (0.3087592021835891, 714, 74, 1, 3), ('leaf', 2), ('leaf', 4), ('leaf', 1), ('leaf', 0), ('leaf', 1), ('leaf', 2), ('leaf', 3), (0.5010351966873706, 549, 8, 0, 1)]
[(0.529757785467128, 775, 28, 4, 2), (0.6010920596040008, 248, 36, 4,

In [455]:
def predictwithprints(row):
    lastcheck=0
    depth=len(splits)-1
    currentclass=None
    for i in range(depth):
        if splits[i][lastcheck][0]=="leaf":
            return currentclass #splits[i][lastcheck][1]

        elif row[splits[i][lastcheck][1]] > splits[i][lastcheck][2]:
            currentclass = splits[i][lastcheck][4]  # Class for right child
            print(currentclass)
            lastcheck=2*i+1

        else:
            currentclass = splits[i][lastcheck][3]  # Class for left child
            print(currentclass)
            lastcheck=2*i
            
   # print("we did it")        
    return currentclass



In [456]:
predictwithprints(test_data_trans.iloc[2])

1
2
2
0
0
3
0
2


2

In [458]:
def predict(row):
    lastcheck=0
    depth=len(splits)-1
    currentclass=None
    for i in range(depth):
        if splits[i][lastcheck][0]=="leaf":
            return currentclass#splits[i][lastcheck][1]

        elif row[splits[i][lastcheck][1]] > splits[i][lastcheck][2]:
            currentclass = splits[i][lastcheck][4]  # Class for right child
            #print(currentclass)
            lastcheck=2*i+1

        else:
            currentclass = splits[i][lastcheck][3]  # Class for left child
            #print(currentclass)
            lastcheck=2*i
            
   # print("we did it")        
    return currentclass



4

In [459]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Generate predictions, passing each row in test_data_trans to predict
predictions = []

for k in range(test_data_trans.shape[0]):
    predictions.append(predict(list(test_data_trans.iloc[k]))) 
    
predictions

# Assuming the last element in each row is the true label
true_labels  = list(test_data_trans.iloc[:, -1])

print(predictions)
print(true_labels)

# Calculate the confusion matrix and accuracy
conf_matrix = confusion_matrix(true_labels, predictions)
accuracy = accuracy_score(true_labels, predictions)

print("Confusion Matrix:")
print(conf_matrix)
print(f"Accuracy: {accuracy * 100:.2f}%")


[2, 0, 2, 3, 0, 2, 0, 0, 0, 0, 3, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 3, 2, 2, 0, 0, 0, 3, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 3, 0, 3, 3, 2, 0, 2, 3, 0, 3, 3, 0, 0, 0, 2, 0, 3, 3, 0, 3, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 2, 0, 0, 0, 2, 2, 0, 2, 0, 3, 3, 2, 0, 3, 3, 0, 2, 2, 2, 2, 0, 2, 2, 3, 0, 3, 3, 0, 3, 3, 0, 3, 0, 3, 3, 0, 2, 0, 0, 3, 0, 0, 0, 0, 0, 3, 2, 0, 3, 2, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 3, 0, 2, 0, 0, 2, 2, 3, 0, 2, 0, 2, 3, 3, 0, 3, 0, 3, 3, 3, 0, 2, 0, 2, 0, 0, 3, 0, 0, 0, 3, 0, 3, 2, 0, 0, 0, 0, 2, 0, 0, 3, 0, 3, 0, 0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 2, 0, 2, 0, 0, 3, 0, 2, 0, 2, 3, 0, 0, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 0, 0, 2, 3, 2, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 2, 3, 0, 2, 0, 0, 0, 0, 0, 2, 3, 3, 3, 3, 0, 0, 0, 2, 2, 0, 3, 2, 3, 0, 0, 0, 2, 3, 0, 0, 3, 0, 3, 0, 0, 3, 2, 3, 0, 0, 2, 2, 3, 2, 0, 0, 3, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 3, 0, 2, 0, 2, 0, 0, 0, 3, 0, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 2, 0, 2, 0, 0, 0, 3, 0, 0, 3, 0, 

In [261]:
import numpy as np

def find_best_split(data):
    best_i = None
    best_column = None
    current_min = 2  # Gini ranges from 0 to 1; initialize with a high value.
    best_split1, best_split2 = None, None
    split_info = None    

    n = len(data)

    for column in data.columns[:-1]:
        # Sort data by the column for efficient split calculations
        sorted_data = data.sort_values(by=column)
        values = sorted_data[column].values
        labels = sorted_data[data.columns[-1]].values

        # Initialize running totals for calculating Gini efficiently
        left_counts = np.zeros(5, dtype=float)
        right_counts = np.bincount(labels, minlength=5).astype(float)
        total_right = len(labels)

        for i in range(len(values) - 1):
            label = labels[i]
            left_counts[label] += 1
            right_counts[label] -= 1

            total_left = i + 1
            total_right -= 1

            # Only consider unique values for splits
            if values[i] == values[i + 1]:
                continue

            gini_left = 1.0 - np.sum((left_counts / total_left) ** 2)
            gini_right = 1.0 - np.sum((right_counts / total_right) ** 2)
            weighted_gini = (gini_left * total_left + gini_right * total_right) / n

            if weighted_gini < current_min:
                best_i = i
                best_column = column
                current_min = weighted_gini
                split_info = [current_min, column, values[i]]

    # Sort the data again by the best column
    sorted_data = data.sort_values(by=best_column)

    # Now split the data into two parts based on best_i
    best_split1 = sorted_data.iloc[:best_i + 1]  # Use iloc to select all columns up to best_i
    best_split2 = sorted_data.iloc[best_i + 1:]  # Use iloc to select all columns after best_i

    # Get the majority class of each split
    Split_majoritys = [
        best_split1[data.columns[-1]].mode().iloc[0],
        best_split2[data.columns[-1]].mode().iloc[0]
    ]

    return split_info, Split_majoritys


In [435]:
# Initialize a counter to store the frequency of each leaf
from collections import Counter
leaf_values = []

# Iterate through all splits and collect leaf values
for sublist in range(len(splits)):
    for item in splits[sublist]:
        if isinstance(item, tuple) and item[0] == 'leaf':  # Check if it is a leaf
            leaf_values.append(item[1])  # Append the leaf value (e.g., 1, 4, etc.)

# Count occurrences of each leaf value using Counter
leaf_counts = Counter(leaf_values)

# Print the leaf counts
print("Leaf values and their counts:")
for leaf, count in leaf_counts.items():
    print(f"Leaf: {leaf}, Count: {count}")

Leaf values and their counts:
Leaf: 2, Count: 31
Leaf: 4, Count: 34
Leaf: 1, Count: 10
Leaf: 0, Count: 24
Leaf: 3, Count: 17
